In [8]:
%run gw_exp.ipynb
%run test.ipynb
# import functions from gw_exp.ipynb

92
1
3
10
92
1
3
10


test_N_1 (__main__.GWInvariantTest) ... ok
test_N_1_0 (__main__.GWInvariantTest) ... ok
test_N_1_1 (__main__.GWInvariantTest) ... ok
test_N_1_2 (__main__.GWInvariantTest) ... ok
test_N_2_0 (__main__.GWInvariantTest) ... ok
test_N_2_1 (__main__.GWInvariantTest) ... ok
test_N_2_2 (__main__.GWInvariantTest) ... ok
test_N_2_3 (__main__.GWInvariantTest) ... ok
test_N_2_4 (__main__.GWInvariantTest) ... ok
test_N_3_3 (__main__.GWInvariantTest) ... 

92
1
3
10


ok
test_N_3_4 (__main__.GWInvariantTest) ... ok
test_N_3_5 (__main__.GWInvariantTest) ... ok
test_N_3_6 (__main__.GWInvariantTest) ... ok
test_N_1 (__main__.TestCP2Recursion) ... ok
test_N_2 (__main__.TestCP2Recursion) ... ok
test_N_3 (__main__.TestCP2Recursion) ... ok
test_N_4 (__main__.TestCP2Recursion) ... ok
test_N_5 (__main__.TestCP2Recursion) ... ok
test_N_6 (__main__.TestCP2Recursion) ... ok
test_N_7 (__main__.TestCP2Recursion) ... ok

----------------------------------------------------------------------
Ran 20 tests in 0.532s

OK


In [16]:
def kontsevich(d):
    """
    Returns the number of rational curves of degree d passing through 3d-1 
    general points in P^2.
    These numbers are also the genus zero Gromov-Witten invariants for P^2. 
    
    EXAMPLES::
    
        sage: kontsevich(3)
        12
        sage: kontsevich(4)
        620
        sage: kontsevich(5)
        87304
    """
    if d == 1:
        return 1
    if d == 2:
        return 1
    f = 0
    for i in range(1,d):
        j = d - i
        a = kontsevich(i)
        b = kontsevich(j)
        f = f + i**2*j*(j*binomial(3*d-4,3*i-2)-i*binomial(3*d-4,3*i-1))*a*b
    return f

def bipart(l):
    """
    Needs this to compute genus zero Gromow-Witten invariants for projective 
    spaces.
    EXAMPLES::
        sage: bipart([1,2])  
        [[[2], [1]], [[1, 2], []], [[], [1, 2]], [[1], [2]]]
        sage: len(bipart([1,2]))
        4
        sage: type(bipart([1,2]))
        <type 'list'>
    """
    if len(l) == 0:
        return [[[],[]]]
    elif len(l) == 1:
        k = l[0]
        return [[[],[k]],[[k],[]]]
    else:
        ll = [l[i] for i in range(len(l)-1)]
        R = bipart(ll)
        R1 = [[R[i][0] + [l[-1]],R[i][1]] for i in range(len(R))]
        R2 = [[R[i][0],R[i][1] + [l[-1]]] for i in range(len(R))]
        return R1 + R2


def GW_invariant(dim, d, l):
        """
        Returns the Gromov-Witten invariants for this projective space.
        INPUT: 
            d is the degree
            l is a list of non-negative integers: for example the number 3
            denotes the class h^3.
        OUTPUT:
            The number of degree d curves meeting some general subvarieties in
            this projective space.
        EXAMPLES::
            Computing the number of lines meeting 4 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(1,[2,2,2,2])
            2
            Computing the number of conics meeting 8 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(2,[2]*8)            
            92
        """
        r = dim
        l.sort()
        l.reverse()
        n = len(l)
        for i in l:
            if i > r:
                return 0
            pass
        if sum(l) != r*d + r + d + n - 3:
            return 0
        elif n == 0 or n == 1:
            return 0
        elif d == 0:
            if n == 3 and sum(l) == r:
                return 1
            else:
                return 0
        elif n == 2:
            if d == 1 and l == [r,r]:
                return 1
            else:
                return 0
        else:
            if l[-1] == 0:
                return 0
            elif l[-1] == 1:
                l.pop(-1)
                return d*GW_invariant(dim, d,l)
            elif r == 2:
                if l == [2]*(3*d-1):
                    return kontsevich(d)
                else:
                    return 0
            else:
                l1 = l[-1] - 1
                l2 = 1
                ll = [l2] + [l[i] for i in range(n-2)] + [l1+l[n-2]]
                res = GW_invariant(dim, d,ll)
                S = bipart([l[i] for i in range(n-3)])
                for s in S:
                    A = s[0]
                    nA = len(A)
                    cA = sum(A)
                    B = s[1]
                    for dA in range(1,d+1):
                        dB = d - dA
                        e = r*dA + r + dA + nA - cA - l1 - l[n-2]
                        if e >= 0 and e <= r:
                            a = GW_invariant(dim, dA,[l1,l[n-2]]+A+[e])
                            b = GW_invariant(dim, dB,[l2,l[n-3]]+B+[r-e])
                            res = res + a*b
                        f = r*dA + r + dA + nA - cA - l1 - l2
                        if f >= 0 and f <= r:
                            x = GW_invariant(dim, dA,[l1,l2]+A+[f])
                            y = GW_invariant(dim, dB,[l[n-2],l[n-3]]+B+[r-f])
                            res = res - x*y
                return res


In [42]:

layer = 0
ogw_d = {}
debugs = []

# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - number of boundary marked points, a non-negative integer
def ogw_invariant(n, b, k, A, layer=-1):
    l = len(A)
    A.sort()    # asceding order
    #A.reverse() # to descending order

    for c_d in A:
        if c_d < 0:
            print("negative classes")
        if c_d > n:
            return 0

    # wall-crossing theorem (Elad)
    if l > 0 and A[0] == -1:
        A.pop(0) # remove the last element        
        return -ogw_invariant(n, A, b, k+1)

    #degree axiom (Elad)
    if 2*sum(A) != (n - 3 + (n+1)*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        # p 595 mcduff 
        #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
        return 0

    # other
    if k == 0 and is_even(b):
        return 0

    # zero axiom
    if b == 0:
        if k == 1 and l == 1 and A[0] == 0:
            ogw_d[str((n, b, k, A))] = -1
            return -1
        else:
            ogw_d[str((n, b, k, A))] = 0
            return 0

    # unit axiom (May)
    # handle all beta = 0 cases
    if l > 0 and A[0] == 0:
        if b == 0 and k == 1 and l == 1:
            ogw_d[str((n, b, k, A))] = -1
            return -1
        else:
            ogw_d[str((n, b, k, A))] = 0
            return 0

    # divisor axiom 
    if l > 0 and A[0] == 1:
        #A.pop(0)
        result = (b/2)*ogw_invariant(n, b, k, A[1:], layer=layer)
        ogw_d[str((n, b, k, A)) + ", divisor"] = result
        return result

    # handle beta = 1, l <= 1 cases
    if b == 1 and k == 2 and A == []:
        ogw_d[str((n, b, k, A))] = 2
        return 2
    if b == 1 and len(A) == 1 and k == 1:
        return 0

    if b == 1 and len(A) == 1 and k == 0:
        return (-1)^((n-1)/2)
        
    """
    if b == 1 and k == 0 and A == [n]:
        ogw_d[str((n, b, k, A))+ ", base case 1"] = (-1)^((n+3)/2) 
        return (-1)^((n+3)/2)

    if b == 1 and k == 1 and A == [(n+1)/2]:
        ogw_d[str((n, b, k, A))] = 0 
        return 0

    if b == 1 and k == 2 and A == []:
        ogw_d[str((n, b, k, A))] = 2
        return 2
        """

    if l >= 2:
        #print("l >= 2")
        #print("compute ", [A[0]-1, A[1] + 1] + A[2:])
        cor_ = ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:], layer=layer)
        cor = cor_
        col = 0
        #print("compute ", [A[0]-1, A[1] + 1] + A[2:], ", b: ", b, ", k: ", k, ", result: ", result)

        #if len(A) == 2:
            #return result

        I = bipart(A[2:]) #bipart([A[i] for i in range(2, l)])
        #print("theorem 10 (a)")
        #print("b: ", b, ", k: ", k)
        #print("original A: ", A)


            # bigger range causes infinite loop
        for b1 in range(1, b):
            #print("b1: ", b1)
            b_ = 0
            if (b - b1) % 2 == 0:
                b_ = (b - b1) / 2
            else:
                continue

            for i in range(0, n+1):
                    #print("i: ", i)
                    #print("ogw1: ", [n-i, A[0]-1] + I2)
                    #print("ogw2: ", [n-i, A[1]] + I2)


                for t in I:
                    I1 = t[0]
                    I2 = t[1]

                    ogw_d["gw: " + str((n, b_, [1, A[1], i] + I1))] = GW_invariant(n, b_, [1, A[1], i] + I1) 
                    ogw_d["gw: " + str((n, b_, [A[0] - 1, i] + I1))] = GW_invariant(n, b_, [1, A[0] - 1, i] + I1)
                    tmp_gw = GW_invariant(n, b_, [1, A[1], i] + I1)
                    tmp_ogw = ogw_invariant(n, b1, k, [n-i, A[0]-1] + I2, layer=layer) 
                    cor += (-1)^(b_*(n+1)/2) *tmp_gw *tmp_ogw
                    if (layer == 1) and False:
                        print("COR: ", cor, ", b_: ", b_, ", i: ", i, " , b1: ", b1, ", gw: ", [1, A[1], i] + I1,\
                            "ogw: ", [n-i, A[0]-1] + I2, ", k: ", k)
                        print("tmp_gw: ", tmp_gw, ", tmp_ogw: ", tmp_ogw)

        for t in I:
            I1 = t[0]
            I2 = t[1]

            # bigger range causes infinite loop
            for b1 in range(1, b):
                #print("b1: ", b1)
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue
                for i in range(0, n+1):
                    col += (-1)^((n+1)*b_/2)* GW_invariant(n, b_, [1, A[0] - 1, i] + I1) * \
                                ogw_invariant(n, b1, k, [n-i, A[1]] + I2, layer=layer)

        ool = 0
        oor = 0

        for t in I:
            I1 = t[0]
            I2 = t[1]
            
            # changing range has no effect on values
            for b1 in range(1, b):
                b2 = b - b1

                # smaller range cause incorrect value
                for k1 in range(0, k+1):
                    k2 = k - k1
                    #print("loop2")
                    ool +=  binomial(k, k1)*ogw_invariant(n, b1, k1, [1, A[0]-1] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[1]] + I2)

                for k1 in range(0, k+1):
                    k2 = k - k1

                    oor += binomial(k, k1) * ogw_invariant(n, b1, k1, [1, A[1]] + I1) \
                                * ogw_invariant(n, b2, k2 + 1, [A[0] - 1] + I2)

        debug_str = "COR_: " + str(cor_) + ", COR: " + str(cor) + " -COL: " + str(-col) + ", -OOR: " + str(-oor) + ", OOL:  " + str(ool)

        if (layer == 1) and debug_str not in debugs:
            debugs.append(debug_str)
            print(debug_str)

        ogw_d[str((n, b, k, A)) + ", theorem 10 (a)"] = cor - col - oor + ool

        return cor - col - oor + ool

    if k >= 2:
        #print("theorem 10 (b) ")
        #print("b: ", b, " , k: ", k, ", A: ", A)
        result = 0

        I = bipart(A)

        for t in I:
            I1 = t[0]
            I2 = t[1]

            # not much effect for n = 7
            for b1 in range(1, b+1):
                if layer == 1:
                    print("b1: ", b1)
                b_ = 0
                if (b + 1 - b1) % 2 == 0:
                    b_ = (b + 1 - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    ogw_d["gw: " + str((n, b_, [1, n, i] + I1))] = GW_invariant(n, b_, [1, n, i] + I1) 
                    tmp1 = 2*(-1)^((n+1)*b_/2)*GW_invariant(n, b_, [1, n, i] + I1) \
                        * ogw_invariant(n, b1, k-1, [n-i] + I2)
                    if layer == 2:
                        print("tmp1: ", tmp1, ", result: ", result + tmp1)
                        print("b_: ", b_, ", gw: ", [1, n, i] + I1)
                        print("b1: ", b1, ", k-1: ", k-1, ", ogw: ", [n-i] + I2)
                    result = result + tmp1                    

        for t in I:
            I1 = t[0]
            I2 = t[1]
            
            for b1 in range(2, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    #print("loop4")
                    #print("binomial1  (",k, ", ", k1, "): ", binomial(k, k2)) 
                    tmp2 = 2*binomial(k-2, k1)*ogw_invariant(n, b1, k1, [1, n] + I1)\
                        * ogw_invariant(n, b2, k2+2, I2)
                    if layer == 1:
                        print("tmp2: ", tmp2, ", result: ", result + tmp2)
                        print("b1: ", b1, ", k1: ", k1, ", ogw1: ", [1, n] + I1)
                        print("b2: ", b2, ", k2+2: ", k2+2, ", ogw2: ", I2)
                    result = result - tmp2
            #print("result1: ", result)

        for t in I:
            I1 = t[0]
            I2 = t[1]

            for b1 in range(1, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    #print("loop5")
                    tmp3 = 2*binomial(k-2, k1)*ogw_invariant(n, b1, k1 + 1, [1] + I1) \
                        *ogw_invariant(n, b2, k2+1, [n]+I2)
                    if layer == 1:
                        print("tmp3: ", tmp3, ", result: ", result - tmp3)
                        print("binomial(k-2, k1): ", binomial(k-2, k1))
                        print("b1: ", b1, ", k1+1: ", k1+1, ", ogw1: ", [1] + I1)
                        print("b2: ", b2, ", k2+1: ", k2+1, ", ogw2: ", [n]+I2) 
                    result = result + tmp3

        ogw_d[str((n, b, k, A)) +", theorem 10 (b)"] = 2*result * (-1)^((n+1)/2)
        return result * (-1)^((n+3)/2)

    print("unexpected situation")
    print("n: ", n, ", b: ", b, ", k: ", k, " A: ", A)
    return 0


In [39]:
try:
    ogw_d = {}
    print("should be -11/32")
    print(ogw_invariant(5, 3, 0, [2]*4+[4]*2))
except RecursionError:
    print("infinite loop")


"""
for key, value in ogw_d.items():
    print(key, ' : ', value)
"""


should be -11/32
-11/32


"\nfor key, value in ogw_d.items():\n    print(key, ' : ', value)\n"

In [19]:
print(gw_invariant(5, 1, [1, 2, 2, 4, 4]))
print(GW_invariant(1,2,[]))

2
0


In [25]:
#print(ogw_invariant(5, 1, 0, [1, 1, 2,2, 2, 2]))

#print(ogw_invariant(5, 3, 0, [2]*10, 1))
print(factorial(0))


1


In [43]:
debugs = []
import sys
original_stdout = sys.stdout # Save a reference to the original standard output

with open('filename.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print('This message will be written to a file.')
    ogw_d = {}
    print("should be -2")
    #print(ogw_invariant(3, 3, 6, []))
    print("should be -255/32") # missing 1/4 somewhere
    print(ogw_invariant(5, 3, 0, [2]*7+[4]*1, 1))
    print("should be 90")
    sys.stdout = original_stdout 
    #print(ogw_invariant(3, 5, 10, []))
    #print(ogw_invariant(3, 3, 4, [3], layer=5))
    #print(ogw_invariant(3, 3, 4,[3]))
layer = 0
    #print(ogw_invariant(3, 2,3, [2], layer=1))
    #print(ogw_invariant(3, 1,2, [0]))
    #print(ogw_invariant(3, 2, 1, [1, 2]))


    #print(ogw_invariant(3, 5, 10, []))

"""
for key, value in ogw_d.items():
    print(key, ' : ', value)
"""

print(ogw_invariant(3, 5, 10, []))



90


In [ ]:
print(gw_invariant(5, ))

In [23]:
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + b*(n+1) + k + 2*l2 - k*n - 2*l2 * A2
    if (m % (2*(A1 - 1))) == 0:
        return m / (2*(A1 - 1))
    else:
        return -1

print(get_l1(5, 1, 0, 0, 2, 4))
print(get_l1(5, 1, 0, 1, 2, 4))
print(get_l1(5, 1, 0, 2, 2, 4))
print(get_l1(5, 1, 0, 3, 2, 4))

print(" ")
print(get_l1(5, 3, 0, 0, 2, 4))
print(get_l1(5, 3, 0, 3, 2, 4))
print(get_l1(5, 3, 0, 2, 2, 4))
print(get_l1(5, 3, 0, 1, 2, 4))
print(get_l1(5, 3, 0, 0, 2, 4))




print("  ")


print(get_l1(7, 1, 0, 0, 2, 6))
print(get_l1(7, 1, 0, 1, 2, 6))

print(get_l1(7, 3, 0, 2, 2, 6))

print(get_l1(7, 3, 0, 1, 2, 6))

print(get_l1(7, 5, 0, 3, 2, 6))


4
1
-2
-5
 
10
1
4
7
10
  
6
1
4
9
7


In [ ]:
print(binomial(1, 1))

1


In [ ]:
def get_i(n, b, k, l, A):
    result = (n - 3 + (n+1)*b + k + 2*l - k*n) - 2*sum(A)
    if result % 2 == 0:
        return result/2
    else:
        return "not result"

print(get_i(3, 2, 9, 1, []))

#gw: sum(l) != n*(d+1) + d + k - 3


def get_k(n, b, A):
    return ((n - 3) + (n+1)*b + 2*len(A) - 2*sum(A))/(n-1)

print(get_k(3, 1, []))
print(binomial(2, 0))
    

-4
2
1


In [35]:
print(509-449)
n = 3

print(ogw_invariant(n, 1, 0, [n]))



60
-1
